In [2]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

# Face Detection Model
- Load the model

In [3]:
face_detection_model = './models/res10_300x300_ssd_iter_140000.caffemodel'
face_detection_proto = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'
# Load models using cv2 dnn
detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
description_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [4]:
!dir images

 Volume in drive C is OS
 Volume Serial Number is 30EA-83F7

 Directory of C:\Face Recogniation App\Face-Recognition\4_machine learning\images

11/21/2021  09:56 PM    <DIR>          .
11/21/2021  09:56 PM    <DIR>          ..
11/21/2021  09:56 PM    <DIR>          Cristiano Ronaldo
11/21/2021  09:56 PM    <DIR>          Elon Musk
11/21/2021  09:56 PM    <DIR>          Lionel Messi
               0 File(s)              0 bytes
               5 Dir(s)  284,372,959,232 bytes free


In [5]:
# consider sample image
img = cv2.imread('./images/Lionel Messi/0001.jpg')
cv2.imshow('Messi',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# step-1: face detection 
image =  img.copy()
h,w = image.shape[:2]
img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop =False)
# set the input
detector_model.setInput(img_blob)
detections = detector_model.forward()

if len(detections) >0:
    i = np.argmax(detections[0,0,:,2]) # consider the face with max confidence score 
    confidence = detections[0,0,i,2]
    if confidence >0.5:
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startx,starty,endx,endy) = box.astype('int')
        # just for reference: draw bounding 
#         img_draw = image.copy()
#         cv2.rectangle(img_draw,(startx,starty),(endx,endy),(0,255,0))

        # step -2: feauture extraction or embedding
        roi = image[starty:endy,startx:endx].copy()
        # get the face descriptors
        faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB = True,crop = True)
        description_model.setInput(faceblob)
        vectors = description_model.forward()
        print(vectors)
        
# cv2.imshow('sample',img)
# cv2.imshow('draw_image',img_draw)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[[-0.04718732  0.04816238 -0.056452   -0.03970181 -0.04276372  0.18095522
  -0.05769016  0.02517243 -0.11138194  0.02155512  0.05111357  0.03491261
   0.13795444 -0.11987813 -0.09860422 -0.01073743 -0.11638693  0.04015779
  -0.04672619  0.14451848  0.1402016  -0.16665484  0.08067489  0.03537824
  -0.05430248  0.09978048 -0.07699743 -0.23078501  0.04759276 -0.03128274
   0.08104167  0.04306683 -0.02533654  0.04038205 -0.02755932  0.06046939
  -0.06568137  0.18582103 -0.00651497 -0.01857661  0.04106737 -0.12262488
  -0.06101248  0.12105981 -0.12960309 -0.15393344  0.19805875 -0.0110271
  -0.17299302  0.00909187 -0.02604421 -0.05663255 -0.07220165  0.05882016
   0.05739852  0.02792654  0.04914029  0.0703489   0.01966189 -0.21799366
  -0.07755356  0.0053152   0.20848563 -0.0866185   0.10708746 -0.03900015
   0.04769563 -0.00710586 -0.01790116  0.04864058 -0.05201461  0.04575072
  -0.04867734  0.06203473  0.06836265 -0.10475557 -0.08750702  0.1026828
   0.03839073  0.0699624  -0.14235938  0